In [1]:
! pip install meerkat-ml
! pip install pydantic<2.0

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.6/3.6 MB 8.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 8.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 18.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.2/53.2 kB 2.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 29.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 79.5/79.5 kB 6.6 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 10.8 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 kB 3.7 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 91.8/91.8 kB 4.5 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 60.8/60.8 kB 5.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 7.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━

/bin/bash: line 1: 2.0: No such file or directory


In [3]:
!pip install "pydantic<2.0"

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 3.1/3.1 MB 10.2 MB/s eta 0:00:00
  Attempting uninstall: pydantic
    Found existing installation: pydantic 2.7.1
    Uninstalling pydantic-2.7.1:
      Successfully uninstalled pydantic-2.7.1


In [ ]:
#!huggingface-cli login

In [4]:
import meerkat as mk
from tqdm import tqdm

In [ ]:
#df = mk.from_huggingface("WillHeld/ACL-OCL-FORK")["train"]

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [6]:
# Set your working directory to a folder in your Google Drive. This way, if your notebook times out,
# your files will be saved in your Google Drive!
import os

# the base Google Drive directory
root_dir = "/content/drive/Shared drives/"

# choose where you want your project files to be saved
project_folder = "CESTA Fellowship: Ottoman Court Histories/Chicago - Topic Modelling/"

def create_and_set_working_directory(project_folder):
  # check if your project folder exists. if not, it will be created.
  if os.path.isdir(root_dir + project_folder) == False:
    os.mkdir(root_dir + project_folder)
    print(root_dir + project_folder + ' did not exist but was created.')

  # change the OS to use your project folder as the working directory
  os.chdir(root_dir + project_folder)

create_and_set_working_directory(project_folder)

In [7]:
df = mk.from_csv("divided_texts.csv")

In [8]:
from transformers import AutoTokenizer, AutoModel
import torch
import torch.nn.functional as F


# tokenizer = AutoTokenizer.from_pretrained('sentence-transformers/all-MiniLM-L6-v2')
# model = AutoModel.from_pretrained('sentence-transformers/all-MiniLM-L6-v2').to("cuda")

tokenizer = AutoTokenizer.from_pretrained("dbmdz/bert-base-turkish-cased")
model = AutoModel.from_pretrained("dbmdz/bert-base-turkish-cased").to("cuda")

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

embs = []

for batch in tqdm(df.batch(batch_size=128), total=len(df) // 128):
  encoded_input = tokenizer(
      batch["text_chunk"].to_numpy().tolist(),
      padding=True, truncation=True, return_tensors='pt'
  ).to("cuda")
  with torch.no_grad():
    model_output = model(**encoded_input)


  emb = mean_pooling(model_output, encoded_input['attention_mask'])
  emb = F.normalize(emb, p=2, dim=1)
  embs.append(emb.to("cpu"))

df["emb"] = torch.concat(embs, axis=0).numpy()

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


tokenizer_config.json:   0%|          | 0.00/60.0 [00:00<?, ?B/s]

config.json:   0%|          | 0.00/385 [00:00<?, ?B/s]

vocab.txt:   0%|          | 0.00/251k [00:00<?, ?B/s]

pytorch_model.bin:   0%|          | 0.00/445M [00:00<?, ?B/s]

62it [04:43,  4.57s/it]


AttributeError: 'Tensor' object has no attribute 'to_numpy'

In [9]:
df["emb"] = torch.concat(embs, axis=0).numpy()

In [23]:
df.write("ottoman-embs.mk")

In [ ]:
print(embs)

In [22]:
!tar -czvf ottoman-embs.mk.tar.gz ottoman-embs.mk/

ottoman-embs.mk/
ottoman-embs.mk/mgr/
ottoman-embs.mk/mgr/blocks/
ottoman-embs.mk/mgr/blocks/135510452346672/
ottoman-embs.mk/mgr/blocks/135510452346672/data.feather
ottoman-embs.mk/mgr/blocks/135510452346672/meta.yaml
tar: ottoman-embs.mk/mgr/blocks/135510452346672/meta.yaml: file changed as we read it
ottoman-embs.mk/mgr/blocks/135507481822496/
ottoman-embs.mk/mgr/blocks/135507481822496/data.npy
ottoman-embs.mk/mgr/blocks/135507481822496/meta.yaml
tar: ottoman-embs.mk/mgr/blocks/135507481822496/meta.yaml: file changed as we read it
ottoman-embs.mk/mgr/columns/
ottoman-embs.mk/mgr/meta.yaml
tar: ottoman-embs.mk/mgr/meta.yaml: file changed as we read it
ottoman-embs.mk/meta.yaml
tar: ottoman-embs.mk/meta.yaml: file changed as we read it


In [ ]:
# now multilingual embeddings

In [26]:
from transformers import BertTokenizer, BertModel
tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')
model = BertModel.from_pretrained("bert-base-multilingual-cased").to("cuda")

def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] #First element of model_output contains all token embeddings
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

embs = []

for batch in tqdm(df.batch(batch_size=128), total=len(df) // 128):
  encoded_input = tokenizer(
      batch["text_chunk"].to_numpy().tolist(),
      padding=True, truncation=True, return_tensors='pt'
  ).to("cuda")
  with torch.no_grad():
    model_output = model(**encoded_input)


  emb = mean_pooling(model_output, encoded_input['attention_mask'])
  emb = F.normalize(emb, p=2, dim=1)
  embs.append(emb.to("cpu"))

df["emb_multilingual"] = torch.concat(embs, axis=0).numpy()

62it [06:53,  6.67s/it]


In [30]:
df

,chunk_filename,text_chunk,year,emb,emb_multilingual
0,ahmedcavid_17901791_ft_clean_1,BİSMÎ'LLÂHİ'R-RAHMÂNİ'R-RAHÎM Hezârân hamd-ı ebedî ol sâni‘-i mükevvenât-ı enfüs u âfâk ve...,1790,"np.ndarray(shape=(768,))","np.ndarray(shape=(768,))"
1,ahmedcavid_17901791_ft_clean_2,ehl-i İslâm'ın pâyını kat* ve ülka-yi Kırım'ı iç-il hükmüne idhâl ile dağdağasız âsûde olmak...,1790,"np.ndarray(shape=(768,))","np.ndarray(shape=(768,))"
2,ahmedcavid_17901791_ft_clean_3,"ric‘at eyledikde, vezîr-i hamiyyet-perver mıkdâr-ı kifâye piyâde ve süvâri guzât-ı şecâ‘at-simâta...",1790,"np.ndarray(shape=(768,))","np.ndarray(shape=(768,))"
3,ahmedcavid_17901791_ft_clean_4,Amire'den donanma tertîb ve Cezâyir'den gelen altı aded sefâ’in ve Tunus'dan gelen süfünün...,1790,"np.ndarray(shape=(768,))","np.ndarray(shape=(768,))"
4,ahmedcavid_17901791_ft_clean_5,rüzgar altında bulunmağla Tersâne donanmasıyla ale'l-gafle Mürted Adası Limanı’na ve Lambro...,1790,"np.ndarray(shape=(768,))","np.ndarray(shape=(768,))"
...,...,...,...,...,...
7904,kefeliibrahim_tevarihitatar_ft_clean_58,Ve bundan ma' dâ emin ve şâdıku’l-va' d pâdişâh idi. Pederi Cengiz Han vefâtndan sonra yine...,1693,"np.ndarray(shape=(768,))","np.ndarray(shape=(768,))"
7905,kefeliibrahim_tevarihitatar_ft_clean_59,"idi. Lâkin, memâlikde olan vilâyet hâkimlerine ziyâdesiyle ruhsat virmekle serkeşnî olub...",1702,"np.ndarray(shape=(768,))","np.ndarray(shape=(768,))"
7906,kefeliibrahim_tevarihitatar_ft_clean_60,ve hasârât iden ammizâdelerinden Deşt-i Kıpçak pâdişâhı Şeyh Bereket Han bin Cuçi Han üzerine...,1711,"np.ndarray(shape=(768,))","np.ndarray(shape=(768,))"
7907,kefeliibrahim_tevarihitatar_ft_clean_61,(s 85) ’l-irş iddi' â itmekle Mâzenderân'a firâr ildirdiler Beşinci Boğa Temur Han Altıncı...,1719,"np.ndarray(shape=(768,))","np.ndarray(shape=(768,))"


In [27]:
df.write("multilingual-embs.mk")

In [28]:
!tar -czvf multilingual-embs.mk.tar.gz multilingual-embs.mk/

multilingual-embs.mk/
multilingual-embs.mk/mgr/
multilingual-embs.mk/mgr/blocks/
multilingual-embs.mk/mgr/blocks/135507096308032/
multilingual-embs.mk/mgr/blocks/135507096308032/data.feather
multilingual-embs.mk/mgr/blocks/135507096308032/meta.yaml
tar: multilingual-embs.mk/mgr/blocks/135507096308032/meta.yaml: file changed as we read it
multilingual-embs.mk/mgr/blocks/135507096320896/
multilingual-embs.mk/mgr/blocks/135507096320896/data.npy
multilingual-embs.mk/mgr/blocks/135507096320896/meta.yaml
tar: multilingual-embs.mk/mgr/blocks/135507096320896/meta.yaml: file changed as we read it
multilingual-embs.mk/mgr/columns/
multilingual-embs.mk/mgr/meta.yaml
tar: multilingual-embs.mk/mgr/meta.yaml: file changed as we read it
multilingual-embs.mk/meta.yaml
tar: multilingual-embs.mk/meta.yaml: file changed as we read it
